In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols

import matplotlib.patheffects as pe
import os
import matplotlib.pyplot as plt
import seaborn as sns
# Seaborn aesthetics
sns.set_context("notebook")
sns.set_theme(style="whitegrid", font_scale=1.4,
              rc={'grid.color': '#ededed'})

from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model
import statsmodels.api as sm
import scipy
from scipy.stats import gaussian_kde
from matplotlib.colors import to_rgba

In [2]:
# Read the neon veg structure measurements!
neon_vst = pd.read_csv("../data/output/NEON_VST_FIRSTMEASUREMENTS.csv")


/tmp/ipykernel_1668079/2758668919.py:2: DtypeWarning: Columns (14,27,28,33,38) have mixed types. Specify dtype option on import or set low_memory=False.
  neon_vst = pd.read_csv("../data/output/NEON_VST_FIRSTMEASUREMENTS.csv")


In [3]:
print(neon_vst.shape)

(137478, 45)


In [4]:
## Clean data
# Use only live trees
neon_vst_filt = neon_vst[neon_vst['plantStatus'].str.contains("Live", na=False)].reset_index(drop=True)
neon_vst_filt = neon_vst_filt[~neon_vst_filt['plantStatus'].str.contains("damage|broken", na=False)].reset_index(drop=True)
# Get average crown diameter
neon_vst_filt["avgCrownDiameter"] = (neon_vst_filt["maxCrownDiameter"] + neon_vst_filt["ninetyCrownDiameter"])/2
neon_vst_filt['crownRadius'] = neon_vst_filt['avgCrownDiameter']/2
# Get vertical crown diameter
neon_vst_filt["verticalDiameter"] = neon_vst_filt["height"] - neon_vst_filt["baseCrownHeight"]
neon_vst_filt['verticalCrownRadius'] = neon_vst_filt['verticalDiameter']/2
# calculate crown volume 
hrad = neon_vst_filt["avgCrownDiameter"]/2
vrad = neon_vst_filt["verticalDiameter"]/2
# appidv_all['crown_vol'] =(4/3) * np.pi * (hrad**2) * (vrad)
neon_vst_filt['crown_vol'] =(4/3) * np.pi * (hrad**2) * ((1/4)*neon_vst_filt["height"])

# Lets just drop duplicates records for now
neon_vst_filt = neon_vst_filt.drop_duplicates(subset=['individualID'])

# Remove data with errors (large DBH, height, etc.)
outlier_trees = ['NEON.PLA.D01.BART.05414', 'NEON.PLA.D01.HARV.05718',
               'NEON.PLA.D01.HARV.05764', 'NEON.PLA.D03.JERC.00993',
               'NEON.PLA.D08.TALL.01932', 'NEON.PLA.D05.TREE.00161',
               'NEON.PLA.D17.SOAP.05687', 'NEON.PLA.D12.YELL.01123'] #outlier we found wiht >160m ht
neon_vst_filt = neon_vst_filt[~neon_vst_filt['individualID'].isin(outlier_trees)].copy().reset_index(drop=True)

# filter for only trees
growthForm_list = ['single bole tree', 'multi-bole tree','small tree'] # 'small tree'
# growthForm_list = ['small tree']
neon_vst_filt = neon_vst_filt[neon_vst_filt['growthForm'].isin(growthForm_list)].copy()
# neon_vst_filt = neon_vst_filt[~neon_vst_filt['siteID'].isin(['PUUM', "GUAN","LAJA"])].copy()


In [5]:
print(neon_vst_filt.shape)
print(neon_vst_filt.columns)


(37097, 50)
Index(['uid', 'namedLocation', 'date', 'eventID', 'domainID', 'siteID',
       'plotID', 'individualID', 'tempStemID', 'tagStatus', 'growthForm',
       'plantStatus', 'stemDiameter', 'measurementHeight',
       'changedMeasurementLocation', 'height', 'baseCrownHeight',
       'breakHeight', 'breakDiameter', 'maxCrownDiameter',
       'ninetyCrownDiameter', 'canopyPosition', 'shape', 'basalStemDiameter',
       'basalStemDiameterMsrmntHeight', 'maxBaseCrownDiameter',
       'ninetyBaseCrownDiameter', 'dendrometerInstallationDate',
       'initialGapMeasurementDate', 'initialBandStemDiameter',
       'initialDendrometerGap', 'dendrometerHeight', 'dendrometerGap',
       'dendrometerCondition', 'bandStemDiameter', 'remarks', 'recordedBy',
       'measuredBy', 'dataEntryRecordID', 'dataQF', 'subplotID', 'taxonID',
       'scientificName', 'easting', 'northing', 'avgCrownDiameter',
       'crownRadius', 'verticalDiameter', 'verticalCrownRadius', 'crown_vol'],
      dtype='objec

In [6]:
# Loop through eah site and get plot statistics (hmax, hmin, dbh, etc.)
df_list = []
for siteid, site_df in neon_vst_filt.groupby("siteID"):
    print(siteid,end="\r")
    # now go through each site
    for plotid,plot_df in site_df.groupby("plotID"):
        # Get plot stats!
        hmax = plot_df['height'].max()
        hmin = plot_df['height'].min()
        hmean = plot_df['height'].mean()
        dmax = plot_df['stemDiameter'].max()
        dmin = plot_df['stemDiameter'].min()
        dmean = plot_df['stemDiameter'].mean()
        crmax = plot_df['avgCrownDiameter'].max()
        crmin = plot_df['avgCrownDiameter'].min()
        crmean = plot_df['avgCrownDiameter'].mean()
        stemcount = len(plot_df.loc[~plot_df['height'].isna()])
        df = pd.DataFrame({"siteID":[siteid],"plotID":[plotid],"stemcount":[stemcount],
                           "hmin":[hmin],"hmax":[hmax],"hmean":[hmean],
                           "dmin":[dmin],"dmax":[dmax],"dmean":[dmean],
                           "crmin":[crmin],"crmax":[crmax],"crmean":[crmean]})
        df_list.append(df)
# Combine dfs
neon_plot_stats = pd.concat(df_list)

In [7]:
neon_plot_stats.shape

(1123, 12)

In [28]:
# Merge with spatial data to save as .shp and visualize!
import geopandas as gpd
fpath = "/data/shared/src/arojas/NEON/data/raw/spatial/All_NEON_TOS_Plots_V9/All_NEON_TOS_Plot_Centroids_V9.shp"
neon_plot_polygons = gpd.read_file(fpath)
neon_plots_stats_merged = pd.merge(neon_plot_polygons, neon_plot_stats.drop(columns=["siteID"]),
                                   how="left",on="plotID")
neon_plots_stats_merged.to_file("../data/output/spatial/NEON_baseplots_firstmsmts_stats")

In [ ]:
# Lets group by siteID and get hmax!

In [31]:
df_list = []
for siteid, site_df in neon_vst_filt.groupby("siteID"):
    print(siteid,end="\r")
    # Get plot stats!
    hmax = site_df['height'].max()
    hmin = site_df['height'].min()
    hmean = site_df['height'].mean()
    
    df = pd.DataFrame({"siteID":[siteid],
                   "hmin":[hmin],"hmax":[hmax],"hmean":[hmean]})
    df_list.append(df)
neon_site_stats = pd.concat(df_list)

In [44]:
# Import neon meta and append!
fpath = "/data/shared/src/arojas/NEON/data/NEON_Field_Site_Metadata_20230309_Env_Vars.csv"
neon_meta = pd.read_csv(fpath)
neon_sites_stats_meta = pd.merge(neon_meta, neon_site_stats.round(3),
                                 how="left",
                                 left_on="field_site_id",right_on="siteID")
# Save to output!
neon_sites_stats_meta.to_csv(fpath,index=False)

In [24]:
######################
## BIOMASS MODELING USING FIA DATABASE OF EQUATIONS
######################

In [8]:
import sqlite3
db_fpath = "/data/shared/src/arojas/Data/nbel/BiomassEqns.db"
conn = sqlite3.connect(db_fpath)
c = conn.cursor()
# sql_query = "SELECT name FROM sqlite_master;"
sql_query = "SELECT * FROM 'BM_ref_species';"
c.execute(sql_query)

In [9]:
import pandas as pd
cols = [column[0] for column in c.description]
bm_ref_species= pd.DataFrame.from_records(data = c.fetchall(), columns = cols)

In [10]:
print(bm_ref_species.shape)
print(bm_ref_species.columns)

(2672, 19)
Index(['spcd', 'common_name', 'genus', 'species', 'variety', 'subspecies',
       'species_symbol', 'sftwd_hrdwd', 'jenkins_spgrpcd',
       'jenkins_sapling_adjustment', 'wood_spgr_greenvol_drywt',
       'bark_spgr_greenvol_drywt', 'mc_pct_green_wood', 'mc_pct_green_bark',
       'wood_spgr_mc12vol_drywt', 'bark_vol_pct', 'green_weight_factor',
       'dry_weight_factor', 'pct_moisture'],
      dtype='object')


In [11]:
bm_ref_species.head(3)

spcd         common_name  genus   species variety subspecies  \
0    10            fir spp.  Abies      spp.                      
1    11  Pacific silver fir  Abies  amabilis                      
2    12          balsam fir  Abies  balsamea                      

  species_symbol sftwd_hrdwd  jenkins_spgrpcd  jenkins_sapling_adjustment  \
0          ABIES           S                3                     0.60817   
1           ABAM           S                3                     0.60193   
2           ABBA           S                3                     0.60817   

   wood_spgr_greenvol_drywt  bark_spgr_greenvol_drywt  mc_pct_green_wood  \
0                      0.36                      0.49           84.00000   
1                      0.40                      0.44           70.00000   
2                      0.33                      0.40          118.54545   

   mc_pct_green_bark  wood_spgr_mc12vol_drywt  bark_vol_pct  \
0           62.43000                     0.38         11.76   
1           63.90909                     0.43         14.00   
2          100.32500                     0.35         12.00   

   green_weight_factor  dry_weight_factor  pct_moisture  
0                 47.2               26.1          81.0  
1                 48.7               28.8          69.2  
2                 51.0               23.6         116.2

In [12]:
db_fpath = "/data/shared/src/arojas/Data/nbel/BiomassEqns.db"
conn = sqlite3.connect(db_fpath)
c = conn.cursor()
# sql_query = "SELECT name FROM sqlite_master;"
sql_query = "SELECT * FROM 'BM_EqForms';"
c.execute(sql_query)
cols = [column[0] for column in c.description]
bm_eq_forms= pd.DataFrame.from_records(data = c.fetchall(), columns = cols)

In [13]:
# sql_query = "SELECT name FROM sqlite_master;"
sql_query = "SELECT * FROM 'BM_EqCoefs';"
c.execute(sql_query)
cols = [column[0] for column in c.description]
bm_eq_coefs= pd.DataFrame.from_records(data = c.fetchall(), columns = cols)

# Filter and clean data
filterbool = (bm_eq_coefs['green_weight']=="N") & (bm_eq_coefs['comp_id']==2)
bm_eq_coefs = bm_eq_coefs.loc[filterbool].copy().reset_index(drop=True)
filterbool = (bm_eq_coefs['biomass_unit']=='KG') & (bm_eq_coefs['ht_unit']=='M')
bm_eq_coefs = bm_eq_coefs.loc[filterbool].reset_index(drop=True)
bm_eq_coefs["R_squre"] = bm_eq_coefs["R_squre"].replace(r'^\s*$', np.nan, regex=True).astype(float)
c.close() # close sql connection
del c

In [14]:
# Get species equations for above ground biomass (use best r2)
df_list=[]
for name,group in bm_eq_coefs.groupby('species_name'):
    if len(group)>1:
        group = group.reset_index(drop=True)
        group_filt = group.loc[[group["R_squre"].idxmax()]]
        df_list.append(group_filt)
    else:
        df_list.append(group)
bm_eq_coefs = pd.concat(df_list)

In [15]:
bm_eq_coefs.shape

(52, 35)

In [16]:
db_fpath = "/data/shared/src/arojas/Data/nbel/BiomassEqns.db"
conn = sqlite3.connect(db_fpath)
conn.text_factory = lambda b: b.decode(errors = 'ignore')
c = conn.cursor()
# sql_query = "SELECT name FROM sqlite_master;"
sql_query = "SELECT * FROM bm_eq_info"
c.execute(sql_query)
cols = [column[0] for column in c.description]
bm_eq_info= pd.DataFrame.from_records(data = c.fetchall(), columns = cols)
c.close()

# Filter boolean for generalized equation models
filterbool=(bm_eq_info['reference_author'].str.contains('Jenk')) & (bm_eq_info['comp_id']==2)
bm_general_eqs = bm_eq_info.loc[filterbool].copy().reset_index(drop=True)
print(bm_general_eqs.shape)

(10, 29)


In [20]:
bm_eq_coefs['species_code'].unique()

array([ 531,  972,  989,  300, 6927,   91,  511,  510,  998,   12,  741,
        743,  543,  762,  837,   95,  986,  740,  701,  129, 7846,  544,
       6284,  400,  105, 8344, 6006,  108, 6008,  833, 7783,  375,  122,
        746,  316,  513,  317, 8355,  318,  514,   71,  999,  541,  951,
        802,   94,  988,  371,  621])

In [18]:
bm_eq_coefs.columns

Index(['eqn_no', 'species_code', 'species_name', 'comp_id', 'eq_form_id', 'a',
       'b', 'c', 'd', 'e', 'dia_type', 'corrected_for_bias', 'bias_correction',
       'r', 'R_squre', 'dia_min', 'dia_max', 'sample_size', 'stump_height',
       'top_dob', 'dia_unit', 'biomass_unit', 'component', 'component_sum',
       'ratio_eq', 'segmented_eq', 'equation_number', 'source', 'notes',
       'created_date', 'ht_unit', 'green_weight', 'ht_type', 'BEQ',
       'modified_date'],
      dtype='object')

In [22]:
# Now that we have a df of species equations and a df of general equations
# Loop through each taxonid, get species code then get the coefs and equation form!
 
vst_taxonids = neon_vst_filt["taxonID"].unique()

for taxonid in vst_taxonids:
    print(taxonid)
    bm_ref_filt = bm_ref_species.loc[bm_ref_species['species_symbol']==taxonid]
    
    
#     bm_eq_coef_filt = bm_eq_coefs['species_code']==bm_ref_filt['']
#     if 
    
    # Get
#     df = pd.DataFrame({"","","a"})

FRPE
PLOC
ACNE2
CELA
ACSA2
QUNI
QULY
DIVI5
ACBA3
LIST2
CAIL2
ACRU
ULAL
QUPA5
ULAM
PRUNU
CATO6
COFO
CACA18
SANI
TRSE6
CAGL8
MORU
CAOV2
MALUS
QUHE
PITA
ASTR
NYSY
QUFA
PODE3
GLTR
CAAQ2
SYTI
MORU2
QUAL
QUPH
nan
QUMI
OSVI
CELTI
NYBI
ILDE
LISI
QUSH
MEAZ
NYAQ2
CARYA
ULMUS
LIJA
CECA4
CAAM2
QUVE
HAVI2
CACO15
CEOC2
JUVI
TADI2
ILVO
PLAQ
TORA2
BENI
LIBE3
QULA3
ULMUSSPP
QUST
FAGR
VIRO3
LITU
BELE
OXAR
HATE3
ACPE
PIST
ACRUR
TSCA
QUMO4
MAFR
CADE12
SAAL5
BEAL2
ILOP
QURU
COFL2
ACSA3
TIAM
MATR
MAAC
PRSE2
PIRU
ABFR
SOAM3
QUCO2
PIRI
KALA
ULRU
HAVI4
RHMA4
FRAM2
RHCO
ROPS
PIVI2
PINUS
PIPU5
AMLA
AEFL
2PLANT
JUNI
PYPU
FRAXI
PIEC2
PHHI2
BOSU2
BUBU
PIAL3
BOURR
CODI8
GYLU
AMEL
EUFO3
KRFE
EXCA
PIUN
GUSA
JACQU
FABACE
BUSI
CROSS
CROTO
HEPU17
PIAC
GUOF
LELE10
DILA10
TAHE
EURH
THSTP
ERAR17
CACY
REUN
PLAL
CRLU2
SIOB
GUKR
JABE
ACFA
PRJU3
REGU
BUMI6
GUOB
ROAC2
CAIN5
SCFR
CAHA9
RAPO2
ZAPO2
SWMA2
SESI3
ACSAS
VACO
CASTA
BEPO
ACER
CACAV
BEPAP
BEPA
PIAB
PRSES
ILEX
PICEA
KALMI
ILMU
AMELA
PIGL
PIMA
BEGL/BENA
PIPA2
BETULA
LIST
Q

In [48]:
bm_ref_species[filterbool]

spcd common_name           genus species variety subspecies  \
2082  8220   monkeypod  Pithecellobium   dulce                      

     species_symbol sftwd_hrdwd  jenkins_spgrpcd  jenkins_sapling_adjustment  \
2082           PIDU           H                8                     0.84031   

      wood_spgr_greenvol_drywt  bark_spgr_greenvol_drywt  mc_pct_green_wood  \
2082                      0.52                      0.53              75.13   

      mc_pct_green_bark  wood_spgr_mc12vol_drywt  bark_vol_pct  \
2082              80.49                     0.58         15.16   

      green_weight_factor  dry_weight_factor  pct_moisture  
2082                 65.9               37.5          75.8

In [27]:
bm_ref_species[["spcd",'species_symbol']]

spcd species_symbol
0       10          ABIES
1       11           ABAM
2       12           ABBA
3       14           ABBR
4       15           ABCO
...    ...            ...
2667  8947           ZIZI
2668     0          2TREE
2669   204           PSME
2670   799            2TB
2671   899            2TB

[2672 rows x 2 columns]

In [25]:
bm_eq_coefs['species_code']

43     531
45     972
68     989
10     300
1     6927
8       91
13     511
14     510
50     998
30      12
0      741
32     743
40     543
44     762
22     837
0       95
69     986
23     740
47     701
18     129
3     7846
41     544
63    6284
17     400
46     400
0      105
4     8344
0     6006
52     108
0     6008
1      833
2     7783
0      375
1      122
31     746
0      316
0      513
38     317
5     8355
36     318
12     514
58      71
0      999
3      999
1      746
39     541
42     951
1      802
2       94
67     988
35     371
19     621
Name: species_code, dtype: int64

In [70]:
# Read in taxon data
fpath = "/data/shared/src/arojas/NEON/data/raw/OS_TAXON_PLANT-20220330T142149.csv"
os_taxon = pd.read_csv(fpath)

/tmp/ipykernel_1552563/1308564196.py:3: DtypeWarning: Columns (11,13,14,24,26,30,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  os_taxon = pd.read_csv(fpath)


In [71]:
os_taxon.columns

Index(['taxonTypeCode', 'taxonID', 'acceptedTaxonID', 'scientificName',
       'scientificNameAuthorship', 'taxonRank', 'vernacularName',
       'taxonProtocolCategory', 'nameAccordingToID', 'nameAccordingToTitle',
       'kingdom', 'subkingdom', 'infrakingdom', 'superdivision', 'division',
       'subdivision', 'infradivision', 'parvdivision', 'superphylum', 'phylum',
       'subphylum', 'infraphylum', 'superclass', 'class', 'subclass',
       'infraclass', 'superorder', 'order', 'suborder', 'infraorder',
       'section', 'subsection', 'superfamily', 'family', 'subfamily', 'tribe',
       'subtribe', 'genus', 'subgenus', 'species', 'subspecies', 'variety',
       'subvariety', 'form', 'subform', 'race', 'stirp', 'morph', 'abberation',
       'unspecified', 'speciesGroup', 'specificEpithet',
       'infraspecificEpithet'],
      dtype='object')

In [85]:
neon_vst_filt['taxonID'].value_counts()

ACRU      2658
PIMA      2216
TSCA      1275
POTR5     1270
MEPO5     1105
          ... 
ALST11       1
JACQU        1
MECL         1
PESA3        1
PIDU         1
Name: taxonID, Length: 360, dtype: int64

In [92]:
os_taxon[os_taxon['taxonID']=="PIMA"][['genus', 'subgenus', 'species', 'subspecies',
                                      'variety',
       'subvariety', 'form', 'subform', 'race', 'stirp', 'morph', 'abberation',
       'unspecified', 'speciesGroup', 'specificEpithet']]

genus subgenus species  subspecies  variety  subvariety  form  subform  \
68585  Picea      NaN     NaN         NaN      NaN         NaN   NaN      NaN   

       race  stirp  morph  abberation  unspecified speciesGroup  \
68585   NaN    NaN    NaN         NaN          NaN          NaN   

      specificEpithet  
68585         mariana

In [93]:
os_taxon[os_taxon['taxonID']=="PIMA"][['vernacularName']]

vernacularName
68585   black spruce